In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from transformers import AutoConfig, AutoTokenizer, AutoModel

In [12]:
df = pd.read_csv('/content/drive/MyDrive/mlops/tamrin1/normalize_divar_dataset.csv')

In [13]:
model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
bert_model = AutoModel.from_pretrained(model_name_or_path)

Some weights of the model checkpoint at HooshvareLab/bert-fa-zwnj-base were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
df["description"] = df['description'].fillna('توضیحات ندارد')
df["sub_title"] = df['sub_title'].fillna('توضیحات ندارد')

In [15]:
title_Bert_encoding = []
for text in tqdm(df["title"]):
    encoded_input = tokenizer(text, return_tensors='pt')
    title_Bert_encoding.append(bert_model(**encoded_input).pooler_output[0].detach().numpy())

title_encoding = [t.astype('float64') for t in title_Bert_encoding]
df.insert(loc=5, column='title_encoding', value=title_Bert_encoding)



description_Bert_encoding = []
for text in tqdm(df["description"]):
    encoded_input = tokenizer(text, return_tensors='pt')
    description_Bert_encoding.append(bert_model(**encoded_input).pooler_output[0].detach().numpy())

description_Bert_encoding = [t.astype('float64') for t in description_Bert_encoding]
df.insert(loc=7, column='description_encoding', value=description_Bert_encoding)


sub_title_Bert_encoding = []
for text in tqdm(df["sub_title"]):
    encoded_input = tokenizer(text, return_tensors='pt')
    sub_title_Bert_encoding.append(bert_model(**encoded_input).pooler_output[0].detach().numpy())

sub_title_Bert_encoding = [t.astype('float64') for t in sub_title_Bert_encoding]
df.insert(loc=10, column='sub_title_encoding', value=sub_title_Bert_encoding)

100%|██████████| 1017/1017 [01:40<00:00, 10.08it/s]


In [16]:
df

,Unnamed: 0.1,Unnamed: 0,index,cat1,cat2,title_encoding,cat3,description_encoding,title,description,sub_title_encoding,url,sub_title,price,size,year,room_number
0,0,0,0,home-kitchen,appliance,"[-0.17308094, -0.3841123, -0.08018968, -0.1692...",washing-machines,"[-0.38248923420906067, -0.6685362458229065, -0...",ماشین لباسشویی دوقلوی کودک,سازنده برند قدیمی ناسیونال,"[0.24733462929725647, -0.12286689877510071, 0....",https://divar.ir/v/ماشین-لباسشویی-دوقلوی-کودک_...,یک ربع پیش در تهران، جوانمرد قصاب,1.850000e+06,0,0,0
1,1,1,1,home-kitchen,appliance,"[-0.07569071, -0.33173707, -0.38010988, 0.4318...",washing-machines,"[0.6938449740409851, -0.5695687532424927, -0.1...",لباسشویی بست BOST,لباسشویی کاملا سالم خدارو شکر مشکلی نداره از ه...,"[-0.3160373270511627, -0.3638947904109955, -0....",https://divar.ir/v/لباسشویی-بست-bost_ماشین-لبا...,نیم ساعت پیش در تهران، آذربایجان,8.000000e+06,0,0,0
2,2,2,2,home-kitchen,appliance,"[0.22114752, -0.5619455, -0.34113258, 0.155819...",washing-machines,"[0.7881287932395935, -0.6777352690696716, -0.4...",ماشین لباسشویی سامسونگ,ماشین لباسشویی سامسونگ کره‌ای اصلی سالم تمیز ب...,"[0.020709101110696793, -0.14797045290470123, -...",https://divar.ir/v/ماشین-لباسشویی-سامسونگ_ماشی...,نیم ساعت پیش در تهران، ابن بابویه,5.500000e+06,0,0,0
3,3,3,3,home-kitchen,appliance,"[0.25543192, -0.5346664, -0.16949041, 0.193233...",washing-machines,"[0.5748929977416992, -0.22399266064167023, 0.0...",موتور لباسشویی هایر,یک عدد موتور لباسشویی برای همه ماشین لباسشویی‌...,"[0.3337443172931671, -0.3063642382621765, -0.4...",https://divar.ir/v/موتور-لباسشویی-هایر_ماشین-ل...,۱ ساعت پیش در تهران، جنت‌آباد جنوبی,5.000000e+05,0,0,0
4,4,4,4,home-kitchen,appliance,"[0.3711403, -0.4896634, 0.6488088, 0.49802044,...",washing-machines,"[0.6281395554542542, -0.8706516623497009, -0.5...",لباسشوئی سامسونگ,ساعات پاسخگویی ۲۰۹ توجه: قیمت کالا مقطوع است و...,"[0.316003680229187, -0.2605590522289276, -0.44...",https://divar.ir/v/لباسشوئی-سامسونگ_ماشین-لباس...,۱ ساعت پیش در تهران، جنت‌آباد شمالی,1.200000e+07,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,1012,1012,1018,vehicles,auto,"[0.61540896, -0.41588452, -0.5935621, -0.03587...",car,"[0.8850921988487244, -0.6237640380859375, -0.5...",سانتافه ۳۵۰۰cc م ۲۰۱۰ فابریک,سانتافه مدل۲۰۱۰ حجم موتور ۳۵۰۰cc رنگ فابریک، م...,"[0.06714774668216705, 0.06084510684013367, -0....",https://divar.ir/v/سانتافه-3500cc-م-2010-فابری...,۳ ساعت پیش در شیراز، فرهنگ شهر,2.400000e+09,0,0,0
1013,1013,1013,1019,vehicles,auto,"[0.4847498, -0.7246818, 0.32035497, 0.09883483...",car,"[0.32556724548339844, 0.19650471210479736, -0....",کوییک دنده‌ای R Plus، مدل ۱۴۰۰,سلام کویک مدل ۱۴۰۰ رنگ مشکی قرمز سالم شاستیهای...,"[0.4964878261089325, -0.1376303732395172, -0.7...",https://divar.ir/v/کوییک-دنده-ای-r-plus-مدل-۱۴...,۳ ساعت پیش در شیراز، شیشه‌گری,3.350000e+08,0,0,0
1014,1014,1014,1020,vehicles,auto,"[0.25469723, -0.3299623, -0.23989576, 0.208270...",car,"[0.06255844980478287, -0.6005983352661133, 0.1...",تیبا هاچبک EX، مدل ۱۴۰۱,سلام درحدصفر ماشین درشهرستان سعادت شهرهست,"[0.07323253154754639, 0.0766276940703392, -0.5...",https://divar.ir/v/تیبا-هاچبک-ex-مدل-۱۴۰۱_سوار...,۳ ساعت پیش در شیراز، ارم,3.800000e+08,0,0,0
1015,1015,1015,1021,vehicles,auto,"[-0.07894157, -0.5292854, -0.20028724, 0.27171...",car,"[-0.020404813811182976, -0.2548191249370575, -...",پژو پارس سال، مدل ۱۴۰۰,با سلام متور زیمنس المان دودی ۲۰ درصد کفی سه ب...,"[0.04773174226284027, 0.20843321084976196, -0....",https://divar.ir/v/پژو-پارس-سال-مدل-۱۴۰۰_سواری...,۳ ساعت پیش در شیراز، کوی زهرا,5.320000e+08,0,0,0


In [17]:
from google.colab import files
df.to_csv('Bert_encoding_Divar.csv', encoding = 'utf-8-sig') 
files.download('Bert_encoding_Divar.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>